# $ Systematic \ Form $
### The following program aims to compute the standard form of an array representing a linear code by repeated Gaussian elimination processes:

### note that the standard form of an array $A \in \mathbb{M}_{n \times m}(\frac{\mathbb{Z}}{2 \mathbb{Z}}) \ for \ (n, m) \in \mathbb{N}²$ is given by : 

#### Assuming that $m$ > $n$ :

\begin{align*}
\begin{bmatrix}
  a_{11} & a_{12} & \dots & a_{1m} \\
  a_{21} & a_{22} & \dots & a_{2m} \\
  \vdots & \vdots & \ddots &  \vdots \\
  a_{n1} & a_{n2} & \dots & a_{nm} \\
\end{bmatrix}
\rightarrow
\begin{bmatrix}
  1 & 0 & \dots & 0 & a_{1n+1} & a_{1n+2} & \dots &  a_{1m}\\
  0 & 1 & \dots & 0 & a_{2n+1} & a_{2n+2} & \dots &  a_{1m}\\
  \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots \\
  0 & 0 & \dots & 1 & a_{nn+1} & a_{nn+2} & \dots &  a_{nm}\\\\
\end{bmatrix}

\end{align*}

### 1) Imports:

In [1]:
import numpy as np

### 2) Definition of the function:

In [2]:
def pivot(matrix, index):
    
    ''' This function takes in a matrix and a given index an exchanges the line
    at that index with the first line bellow it that has non zero element at
    that given index's column '''
    
    elements_in_column = list(matrix[index + 1:, index])    # listing all the entries bellow the diagonal matrix[index, index] to find the first non zero one
    relative_index = next((ind for ind, value in enumerate(elements_in_column) if value != 0), None)    # getting the relative index of the first non zero element inside the "element_in_column" list
    
    first_non_zero_index = index + relative_index + 1 # calculating the index of the first non zero element bellow the diagonal entry matrix[index, index] in the column
                
    matrix[index], matrix[first_non_zero_index] = matrix[first_non_zero_index].copy(), matrix[index].copy() # swapping the lines of the matrix to bring the pivot to the desired place
    
    return matrix

In [9]:
def binary_gaussian_ellimination(matrix):
    
    ''' This function takes in a matrix as input and procedes with the gaussian
    ellimination process to triangularize the matrix in the finite field Z/2Z with
    partial pivoting '''
    ''' Note that this function yields exactly the standard form of the generator
    matrix if the latter is not square '''

    for i in range(0, matrix.shape[0] - 1): # itterating through all diagonal elements
    
        if matrix[i, i] == 0:
            matrix = pivot(matrix, i) # using partial pivoting if the diagonal element is nought
            
        else:   # eliminating each column using the diagonal element as a pivot and only looking for entries diffrent from 0 to optimize the runtime
            for j in range(i + 1, matrix.shape[0]): 
                if matrix[j, i] == 1:
                    matrix[j] = (matrix[j] + matrix[i])%2

    return matrix

In [19]:
def systematic_form(generator_matrix):
    
    ''' This function takes in a generator_matrix and computes its standard form by
    taking in consideration its dimention and transposing the matrix when the latter
    is up standing before calculating, it also diagonalize the matrix when the latter
    is square returning the identity matrix'''
    
    if generator_matrix.shape[0] < generator_matrix.shape[1]: # if the matrix lays flat then we compute the systematic form directly
        return binary_gaussian_ellimination(generator_matrix)
    
    elif generator_matrix.shape[0] == generator_matrix.shape[1]:    # if the matrix is square we diagonalize it
        generator_matrix = binary_gaussian_ellimination(generator_matrix)   # first triangularizing the matrix 
        
        index_list = list(range(-generator_matrix.shape[0], -1))    # list of the indexes of the lines going from bottom to top
        index_list.reverse()    # reversing the list to get the right order

        for i in index_list:    # iterating through all the lines of the matrix from bottom to top
            index_of_non_zero_element_over_diagonal = [index for index, value in enumerate(generator_matrix[i]) if value != 0]  # list containing all indexes of the non zero entries of the line
            
            for index in index_of_non_zero_element_over_diagonal[1:]:   # iterating through all non zero entries strictly over the diagonal and using the line of the same index in order to get rid of that element since the latter contains zeros besides on that index
                generator_matrix[i] = (generator_matrix[i] + generator_matrix[index])%2
        
        return generator_matrix
    
    else:   # case where the matrix stands up
        generator_matrix = np.transpose(generator_matrix)   # transposing the matrix
        
        return np.transpose(systematic_form(generator_matrix))
        

## Testing Cell:

In [21]:
# Testing for a flat matrix:

array = np.array(
                [[1,0,0,1,0,1],
                 [0,1,0,1,1,0],
                 [0,0,1,0,1,1]])

standard_form = systematic_form(array)
print(standard_form)

[[1 0 0 1 0 1]
 [0 1 0 1 1 0]
 [0 0 1 0 1 1]]


In [22]:
# Testing for a square matrix:

array = np.array(
                 [[1,0,0,1,0,1]
                 ,[0,1,0,1,1,0]
                 ,[0,0,1,0,1,1]
                 ,[0,0,0,1,0,1]
                 ,[0,0,1,0,0,0]
                 ,[0,0,0,0,1,0]])

standard_form = systematic_form(array)
print(standard_form)

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [23]:
# Testing for a standing matrix:

array = np.array(
                [[1,0,0,1,0,1],
                 [0,1,0,1,1,0],
                 [0,0,1,0,1,1]])

standard_form = systematic_form(np.transpose(array))
print(standard_form)

[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [0 1 1]
 [1 0 1]]
